# 🎌 Anime 4K Upscale

In [ ]:
#@title ⚙️ CẤU HÌNH
INPUT_DIR = "/content/drive/MyDrive/Movie/Input" #@param {type:"string"}
OUTPUT_DIR = "/content/drive/MyDrive/Movie/Output" #@param {type:"string"}
MODE = "anime_4k" #@param ["anime_4k", "anime_fast", "ultra_fast"]
WORKER_ID = 1 #@param {type:"integer"}
TOTAL_WORKERS = 1 #@param {type:"integer"}

In [ ]:
#@title 🚀 CHẠY
import os, glob, subprocess, json, time, pickle, site
from datetime import timedelta

os.chdir('/content')

from google.colab import drive
drive.mount('/content/drive')

print("🖥️ GPU:")
!nvidia-smi --query-gpu=name,memory.free --format=csv

# ====== PATCH BASICSR ======
print("\n🔧 Patching...")
for sp in site.getsitepackages() + ['/usr/local/lib/python3.12/dist-packages']:
    pf = os.path.join(sp, 'basicsr/data/degradations.py')
    if os.path.exists(pf):
        with open(pf, 'r') as f: c = f.read()
        if 'functional_tensor' in c:
            c = c.replace('from torchvision.transforms.functional_tensor import rgb_to_grayscale', 'from torchvision.transforms.functional import rgb_to_grayscale')
            with open(pf, 'w') as f: f.write(c)
            print(f"   ✓ basicsr patched")

# ====== INSTALL ======
print("\n📦 Setup...")
os.chdir('/content')
ESRGAN = '/content/Real-ESRGAN'
MODEL = f'{ESRGAN}/weights/realesr-animevideov3.pth'

if not os.path.exists(MODEL):
    !rm -rf {ESRGAN}
    !git clone https://github.com/xinntao/Real-ESRGAN.git
    os.chdir(ESRGAN)
    !pip install facexlib gfpgan -q
    !pip install -r requirements.txt -q
    !python setup.py develop 2>&1 | tail -1
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth -P weights/ -q

# ====== PATCH REAL-ESRGAN (fix nb_frames) ======
inference_file = f'{ESRGAN}/inference_realesrgan_video.py'
if os.path.exists(inference_file):
    with open(inference_file, 'r') as f: code = f.read()
    
    # Fix: handle missing nb_frames
    old_code = "ret['nb_frames'] = int(video_streams[0]['nb_frames'])"
    new_code = '''    # Handle missing nb_frames
    if 'nb_frames' in video_streams[0]:
        ret['nb_frames'] = int(video_streams[0]['nb_frames'])
    else:
        # Calculate from duration and fps
        duration = float(ret.get('duration', 0))
        fps_parts = video_streams[0].get('r_frame_rate', '24/1').split('/')
        fps = float(fps_parts[0]) / float(fps_parts[1]) if len(fps_parts) == 2 else 24
        ret['nb_frames'] = int(duration * fps) if duration else 10000'''
    
    if old_code in code:
        code = code.replace(old_code, new_code)
        with open(inference_file, 'w') as f: f.write(code)
        print("   ✓ Real-ESRGAN patched (nb_frames fix)")

print("✅ Ready!")

# Config
MODES = {
    'anime_4k': {'tile': 192, 'tile_pad': 24, 'crf': 22, 'preset': 'fast'},
    'anime_fast': {'tile': 128, 'tile_pad': 16, 'crf': 24, 'preset': 'veryfast'},
    'ultra_fast': {'tile': 64, 'tile_pad': 8, 'crf': 28, 'preset': 'ultrafast'}
}
cfg = MODES[MODE]
PROGRESS = f"/content/drive/MyDrive/Movie/.progress_{WORKER_ID}.pkl"

# SCAN
print(f"\n🔍 Scanning...")
def get_info(p):
    try:
        r = subprocess.run(['ffprobe','-v','quiet','-print_format','json','-show_format','-show_streams',p], capture_output=True, text=True, timeout=30)
        d = json.loads(r.stdout)
        v = next((s for s in d.get('streams',[]) if s.get('codec_type')=='video'), {})
        return {'w': int(v.get('width',0)), 'h': int(v.get('height',0)), 'dur': float(d.get('format',{}).get('duration',0))}
    except: return {'w':0,'h':0,'dur':0}

done = set()
if os.path.exists(PROGRESS):
    with open(PROGRESS, 'rb') as f: done = pickle.load(f)

all_files = []
if os.path.exists(INPUT_DIR):
    for root, dirs, files in os.walk(INPUT_DIR):
        for f in files: all_files.append(os.path.join(root, f))

VIDEO_EXT = ('.mp4', '.mkv', '.avi', '.mov', '.webm')
all_vids = sorted([f for f in all_files if f.lower().endswith(VIDEO_EXT)])
my_vids = [v for i,v in enumerate(all_vids) if (i % TOTAL_WORKERS) == (WORKER_ID - 1)]

to_process = []
for p in my_vids:
    name = os.path.basename(p)
    out = os.path.join(OUTPUT_DIR, name.rsplit('.',1)[0] + '_4K.mkv')
    info = get_info(p)
    if name in done or (os.path.exists(out) and os.path.getsize(out) > 10*1024*1024):
        print(f"⏭️ {name}")
    elif info['w'] >= 3800:
        print(f"⏭️ [4K] {name}")
    elif info['w'] > 0:
        to_process.append({'path': p, 'name': name, 'info': info})
        print(f"✅ {name} ({info['w']}x{info['h']})")

print(f"📊 {len(to_process)} videos")

# PROCESS
if to_process:
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.makedirs('/content/temp', exist_ok=True)
    print(f"\n🚀 Mode: {MODE}\n")
    
    for idx, item in enumerate(to_process, 1):
        p, name, info = item['path'], item['name'], item['info']
        out = os.path.join(OUTPUT_DIR, name.rsplit('.',1)[0] + '_4K.mkv')
        temp = f'/content/temp/{idx}.mp4'
        
        print(f"🎌 [{idx}/{len(to_process)}] {name}")
        start = time.time()
        
        os.chdir(ESRGAN)
        ret = os.system(f'python inference_realesrgan_video.py -n realesr-animevideov3 -i "{p}" -o "{temp}" -s 2 --tile {cfg["tile"]} --tile_pad {cfg["tile_pad"]} --suffix "" --fp32')
        
        if not os.path.exists(temp):
            print(f"❌ Upscale failed\n")
            continue
        
        os.system(f'ffmpeg -y -i "{p}" -i "{temp}" -map 1:v -map 0:a? -map 0:s? -c:v libx265 -crf {cfg["crf"]} -preset {cfg["preset"]} -tag:v hvc1 -c:a copy -c:s copy -map_metadata 0 -map_chapters 0 "{out}" -loglevel warning -stats')
        
        done.add(name)
        with open(PROGRESS, 'wb') as f: pickle.dump(done, f)
        if os.path.exists(temp): os.remove(temp)
        print(f"✅ {timedelta(seconds=int(time.time()-start))} | {os.path.getsize(out)/(1024**3):.2f}GB\n")
    
    print(f"🏁 XONG! {OUTPUT_DIR}")
else:
    print("✅ Không có video!")